In [11]:
! pip3 install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [12]:
import fitz  # PyMuPDF
import requests
import os, json
pdf_url = "https://people.eecs.berkeley.edu/~vazirani/algorithms/chap0.pdf"
pdf_filename = "chap0.pdf"
response = requests.get(pdf_url)
if response.status_code == 200:
    with open(pdf_filename, 'wb') as pdf_file:
        pdf_file.write(response.content)
else:
    print("Failed to download the PDF file")
    exit()

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

pdf_text = extract_text_from_pdf(pdf_filename)

In [13]:
pdf_json = {
    "content": pdf_text
}


In [14]:
json_filename = "chap0.json"
with open(json_filename, 'w') as json_file:
    json.dump(pdf_json, json_file, indent=4)


In [15]:
os.remove(pdf_filename)

print(f"PDF content has been converted to JSON and saved to {json_filename}")


PDF content has been converted to JSON and saved to chap0.json


In [16]:
with open('chap0.json', 'r') as file:
    data = json.load(file)

In [17]:
def split_text(text, max_length=3000):
    sentences = text.split('.')
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) <= max_length:
            chunk += sentence + '.'
        else:
            chunks.append(chunk)
            chunk = sentence + '.'
    chunks.append(chunk)  # add the last chunk
    return chunks

chunks = split_text(data['content'])


In [18]:
! pip3 install torch

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [19]:
! pip3 install accelerate

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [23]:
! pip3 install "vllm>=0.4.3"



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 823 kB 5.9 MB/s eta 0:00:01
  distutils: /private/var/folders/h1/yw7_75395vg1f2_pcmjk7kqw0000gq/T/pip-build-env-h3r2kg0m/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/h1/yw7_75395vg1f2_pcmjk7kqw0000gq/T/pip-build-env-h3r2kg0m/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/h1/yw7_75395vg1f2_pcmjk7kqw0000gq/T/pip-build-env-h3r2kg0m/normal'
  distutils: /private/var/folders/h1/yw7_75395vg1f2_pcmjk7kqw0000gq/T/pip-build-env-h3r2kg0m/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/h1/yw7_75395vg1f2_pcmjk7kqw0000gq/T/pip-build-env-h3r2kg0m/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = 

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")


Loading checkpoint shards: 100%|██████████| 4/4 [00:17<00:00,  4.43s/it]
Some parameters are on the meta device device because they were offloaded to the disk.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
def get_answer_from_chunk(chunk, question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question},
        {"role": "user", "content": chunk}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

# Example question
question = "What is the main topic of the PDF?"

# Iterate through chunks to get answers
for chunk in chunks:
    answer = get_answer_from_chunk(chunk, question)
    print(answer)
#final_answer = get_answer_from_chunk(answer, question)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/Users/vxhan/Library/Python/3.9/lib/python/site-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(


The main topic of the PDF excerpt appears to be the significance and importance of algorithms in the advancement of technology and human progress. The text discusses the impact of two major innovations - the printing press and the decimal number system - and argues that while the printing press facilitated the spread of literacy and knowledge, it was the development of efficient algorithms that enabled complex computations and paved the way for the digital age. The excerpt highlights the role of algorithms in arithmetic operations, scientific calculations, and their eventual integration into the design of computers, emphasizing their pivotal role in modern society.
The main topic of the provided text is the exploration of the Fibonacci sequence, its historical significance, and the introduction of an algorithm to compute the nth Fibonacci number. The text emphasizes the importance of the Fibonacci sequence across various fields and highlights the inefficiency of a naive recursive appro

KeyboardInterrupt: 